# **Plotting ICESat-2 and CryoSat-2 Freeboards**


<div align="center"><img width="40%"  src='https://raw.githubusercontent.com/nsidc/NSIDC-Data-Tutorials/is2-cryo2/notebooks/ICESat-2_Cloud_Access/img/icesat2-cryosat2.png'/></div>

## Note

This tutorial was originally used for a data access workshop during the Cryo2Ice symposium in September 2024.  Content represents a snapshot in time, and will not be updated. 

### **Credits**
This notebook was created by Mikala Beig and Andy Barrett, NSIDC

### **Learning Objectives** 

1. use `earthaccess` to search for ICESat-2 ATL10 data using a spatial filter
2. open cloud-hosted files using direct access to the ICESat-2 S3 bucket; 
3. use cs2eo script to download files into your hub instance
3. load an HDF5 group into an `xarray.Dataset`;  
4. visualize freeboards using `hvplot`.
5. map the locations of ICESat-2 and CryoSat-2 freeboards using `cartopy`

### **Prerequisites**

1. An EC2 instance in the us-west-2 region. **NASA cloud-hosted data are in Amazon Region us-west2. So you also need an EC2 instance in the us-west-2 region.** .
2. An Earthdata Login is required for data access. If you don't have one, you can register for one [here](https://urs.earthdata.nasa.gov/).
3. Experience using cs2eo to query for coincident data.
4. A cs2eo download script for CryoSat-2 data.


### **Tutorial Steps**

#### Query for coincident ICESat-2 and CryoSat-2 data

Using the cs2eo coincident data explorer, query for ATL10 and CryoSat-2, L2, SAR, POCA, Baseline E data products using a spatial and temporal filter.

**Download the basic result metadata and the raw access scripts.**  Upload the ESA download script (SIR_SAR_L2_E_download_script.py) into the folder from which you are running this notebook.

![cs2eo query](./img/ATL10_CS2_L2_SAR_query_med.png)

#### Import Packages

In [ ]:
import os
import platform
from ftplib import FTP
import sys


# For searching and accessing NASA data
import earthaccess

# For reading data, analysis and plotting
import xarray as xr
import hvplot.xarray

# For nice printing of python objects
import pprint  

# For plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#downloading files using cs2eo script
from SIR_SAR_L2_E_download_script import download_files

## use your own email here
user_email = 'your email here'
path = './data/'

#### Download CryoSat-2 data to your hub instance

Copy the list of ESA files from within SIR_SAR_L2_E_download_script.py 

In [ ]:
esa_files = ['SIR_SAR_L2/2019/12/CS_LTA__SIR_SAR_2__20191227T110305_20191227T111751_E001.nc', 
             'SIR_SAR_L2/2020/03/CS_LTA__SIR_SAR_2__20200329T163208_20200329T164044_E001.nc', 
             'SIR_SAR_L2/2020/01/CS_LTA__SIR_SAR_2__20200114T203033_20200114T204440_E001.nc', 
             'SIR_SAR_L2/2019/11/CS_LTA__SIR_SAR_2__20191103T134759_20191103T135125_E001.nc', 
             'SIR_SAR_L2/2020/02/CS_LTA__SIR_SAR_2__20200204T191657_20200204T192558_E001.nc', 
             'SIR_SAR_L2/2019/12/CS_LTA__SIR_SAR_2__20191216T215645_20191216T220909_E001.nc', 
             'SIR_SAR_L2/2020/03/CS_LTA__SIR_SAR_2__20200315T065755_20200315T071241_E001.nc', 
             'SIR_SAR_L2/2019/10/CS_LTA__SIR_SAR_2__20191030T135252_20191030T135600_E001.nc', 
             'SIR_SAR_L2/2020/02/CS_LTA__SIR_SAR_2__20200219T081800_20200219T083303_E001.nc', 
             'SIR_SAR_L2/2020/01/CS_LTA__SIR_SAR_2__20200110T203717_20200110T204612_E001.nc', 
             'SIR_SAR_L2/2020/04/CS_LTA__SIR_SAR_2__20200409T053748_20200409T054151_E001.nc', 
             'SIR_SAR_L2/2020/04/CS_LTA__SIR_SAR_2__20200413T053254_20200413T053659_E001.nc', 
             'SIR_SAR_L2/2020/02/CS_LTA__SIR_SAR_2__20200208T191154_20200208T192117_E001.nc', 
             'SIR_SAR_L2/2020/03/CS_LTA__SIR_SAR_2__20200319T065300_20200319T070802_E001.nc', 
             'SIR_SAR_L2/2020/03/CS_LTA__SIR_SAR_2__20200304T175209_20200304T180102_E001.nc', 
             'SIR_SAR_L2/2019/11/CS_LTA__SIR_SAR_2__20191128T122800_20191128T123212_E001.nc', 
             'SIR_SAR_L2/2019/10/CS_LTA__SIR_SAR_2__20191009T150801_20191009T151142_E001.nc', 
             'SIR_SAR_L2/2019/11/CS_LTA__SIR_SAR_2__20191121T231659_20191121T232817_E001.nc', 
             'SIR_SAR_L2/2020/02/CS_LTA__SIR_SAR_2__20200215T082253_20200215T083741_E001.nc', 
             'SIR_SAR_L2/2020/01/CS_LTA__SIR_SAR_2__20200121T094259_20200121T095800_E001.nc', 
             'SIR_SAR_L2/2019/10/CS_LTA__SIR_SAR_2__20191005T151255_20191005T151621_E001.nc', 
             'SIR_SAR_L2/2020/04/CS_LTA__SIR_SAR_2__20200427T150701_20200427T151544_E001.nc', 
             'SIR_SAR_L2/2019/10/CS_LTA__SIR_SAR_2__20191024T004201_20191024T005059_E001.nc', 
             'SIR_SAR_L2/2020/03/CS_LTA__SIR_SAR_2__20200308T174708_20200308T175621_E001.nc', 
             'SIR_SAR_L2/2020/04/CS_LTA__SIR_SAR_2__20200402T162707_20200402T163602_E001.nc']

Download the CryoSat-2 files into your hub instance by calling the download_files function you imported from the script.

In [ ]:
download_files(user_email, esa_files)

Stashing the files in a data folder to keep our notebook directory less cluttered.

In [ ]:
!mv CS_LTA__SIR*.nc data

#### Use `earthaccess` for direct S3 access of ATL10

First we authenticate using `earthaccess`

In [ ]:
auth = earthaccess.login()

Locate the ATL10 urls in the ATL10 download script provided by cs2eo (ATL10_V6_download_script.py).  Copy the cloudfront urls from the script into a list, as below. 

In [ ]:
# url_list from ATL10 download script provided by cs2eo
cloudfront_urls = ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/12/16/ATL10-01_20191216181242_12350501_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/11/03/ATL10-01_20191103095257_05730501_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/04/02/ATL10-01_20200402125328_01070701_006_02.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/10/05/ATL10-01_20191005111655_01310501_006_02.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/02/04/ATL10-01_20200204154125_06100601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/03/08/ATL10-01_20200308140907_11130601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/02/08/ATL10-01_20200208153304_06710601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/10/19/ATL10-01_20191019210040_03510501_006_02.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/10/27/ATL10-01_20191027204400_04730501_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/02/15/ATL10-01_20200215044202_07710601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/02/19/ATL10-01_20200219043343_08320601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/03/19/ATL10-01_20200319030948_12740601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/10/23/ATL10-01_20191023205221_04120501_006_02.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/04/27/ATL10-01_20200427113753_04880701_006_02.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/12/27/ATL10-01_20191227071321_00090601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/03/29/ATL10-01_20200329130147_00460701_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/03/15/ATL10-01_20200315031809_12130601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/01/10/ATL10-01_20200110165702_02290601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/10/02/ATL10-01_20191002202520_00910501_006_02.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/10/30/ATL10-01_20191030100118_05120501_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/10/09/ATL10-01_20191009110835_01920501_006_02.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/04/09/ATL10-01_20200409020227_02070701_006_02.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/01/21/ATL10-01_20200121055741_03900601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/01/14/ATL10-01_20200114164843_02900601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/11/21/ATL10-01_20191121192822_08540501_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/03/04/ATL10-01_20200304141724_10520601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/01/25/ATL10-01_20200125054923_04510601_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/11/28/ATL10-01_20191128083720_09540501_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2020/04/13/ATL10-01_20200413015408_02680701_006_02.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/12/12/ATL10-01_20191212182101_11740501_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/11/17/ATL10-01_20191117193643_07930501_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/12/02/ATL10-01_20191202082858_10150501_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/12/23/ATL10-01_20191223072141_13350501_006_01.h5', 
                   'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2019/10/31/ATL10-01_20191031190122_05330501_006_01.h5']

We use earthaccess.open() to directly access the ATL10 files within their S3 bucket.  earthaccess.open() creates a file-like object, which is required because AWS S3 uses object storage, and we need to create a virtual file system to work with the HDF5 library.

In [ ]:
icesat2_files = earthaccess.open(cloudfront_urls)

We can use xarray to examine the contents of our files (one group at a time).

In [ ]:
ds_is2 = xr.open_dataset(icesat2_files[1], group='gt2r/freeboard_segment')
ds_is2

And we can use hvplot to plot one of the variables within that group.

In [ ]:
ds_is2['beam_fb_height'].hvplot(kind='scatter', s=2)

#### Open and plot downloaded CryoSat-2 data 

We need a list of the downloaded CryoSat-2 files.

In [ ]:
downloaded_files = os.listdir(path)
downloaded_files

We use xarray to access the contents of our netcdf file.  In this case, we are not "streaming" data from an S3 bucket, but are accessing the data locally.

In [ ]:
ds_cs2 = xr.open_dataset(path + downloaded_files[0])
ds_cs2

In [ ]:
ds_cs2['radar_freeboard_20_ku'].hvplot(kind='scatter', s=2)

#### Plot ICESat-2 and CryoSat-2 Freeboards on same map

Here we're plotting one file from each data set to save time.

In [ ]:
projection = ccrs.Stereographic(central_latitude=90.,
                                central_longitude=-45.,
                                true_scale_latitude=70.)
extent = [-2500000.000, 2500000., -2500000., 2500000.000]


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection=projection)
ax.set_extent(extent, projection)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.LAND)
ax.gridlines(draw_labels=True)

vmin = 0.
vmax = 1.

# Plot Cryosat freeboard
cs2_img = ax.scatter(ds_cs2.lon_poca_20_ku, ds_cs2.lat_poca_20_ku, 5,
                        c=ds_cs2.radar_freeboard_20_ku, 
                        vmin=vmin, vmax=vmax,  # Set max and min values for plotting
                        cmap='Reds',   # shading='auto' to avoid warning
                        transform=ccrs.PlateCarree())  # coords are lat,lon but map if NPS 

# Plot IS2 freeboard 
is2_img = ax.scatter(ds_is2.longitude, ds_is2.latitude, 5,
                     c=ds_is2.beam_fb_height, 
                     vmin=vmin, vmax=vmax,  
                     cmap='Purples', 
                     transform=ccrs.PlateCarree())

# Add colorbars
fig.colorbar(cs2_img, label='Cryosat-2 Radar Freeboard (m)')
fig.colorbar(is2_img, label='ICESat-2 Lidar Freeboard (m)')